In [1]:
from urllib.request import urlopen
from html.parser import HTMLParser
import os
import wget

In [2]:
class MyHTMLParser(HTMLParser):
    def __init__(self, predicate):
        super().__init__()
        self.journals = []
        self.state = False
        self.predicate = predicate
        
    def handle_starttag(self, tag, attrs):
        self.state = tag == 'a'
        
    def handle_data(self, data):
        text = data.strip()
        if self.state and len(text) > 0 and self.predicate(text):
            self.journals.append(data)

class PDFHTMLParser(HTMLParser):
    def __init__(self, predicate):
        super().__init__()
        self.journals = []
        self.predicate = predicate
        
    def handle_data(self, data):
        text = data.strip()
        if self.predicate(text):
            self.journals.append(data)

# Выгружение всех журналов

In [105]:
url_root = "https://socionet.ru/~cyrcitec/newr-pra/mprapa/"
html_root = urlopen(url_root).read().decode()

journals_catalogue = MyHTMLParser(lambda x: x.endswith('.xml'))
journals_catalogue.feed(html_root)

In [106]:
xml_root = "http://no-xml.socionet.ru/~cyrcitec/newr-pra/mprapa/"

In [107]:
len(journals_catalogue.journals)

39722

In [108]:
from lxml import etree

In [109]:
def parseXML(xml):
    """
    Парсинг XML
    """
    handle, pdf_link, json_link = None, None, None
    
    if xml.decode().count('\n') <= 17:
        return handle, pdf_link, json_link
    root = etree.fromstring(xml)
    
    for appt in root.getchildren():
        handle = appt.attrib.get('handle') 
        for elem in appt.getchildren():
            if elem.text:
                json_link = elem.attrib.get('rich')
                for t in elem:
                    pdf_link = t.attrib.get('data')
    return handle, pdf_link, json_link

In [110]:
parseXML(urlopen(''.join([xml_root, journals_catalogue.journals[1]])).read())

('repec:pra:mprapa:5',
 'http://no-xml.socionet.ru/~cyrcitec/pdf-cache/mpra.ub.uni-muenchen.de/5/1/MPRA_paper_5.pdf',
 'http://no-xml.socionet.ru/~cyrcitec/j-pra/mprapa/5.json')

# Из каждого журнала выбираем статью (например, первую) и ищем для нее PDF

In [111]:
try:
    os.mkdir('articles')
except:
    pass

In [112]:
url_prefix = "http://no-xml.socionet.ru/~cyrcitec/"
url_prefix_len = len(url_prefix)

In [113]:
import numpy as np
np.random.seed(42)

In [114]:
for journal in np.random.choice(journals_catalogue.journals, size=250):
    print(journal)
    try:
        journal_id, journal_link_pdf, journal_link_json = parseXML(urlopen(''.join([xml_root, journal])).read())
        print(journal_id, journal_link_pdf, journal_link_json)
        if (journal_id is None) or (journal_link_pdf is None) or (journal_link_json is None):
            continue
    except:
        continue
    
    try:
        os.mkdir(f'articles/{journal_id}')
    except:
        pass
    try:
        wget.download(journal_link_pdf, out=f'articles/{journal_id}/{journal_id}:orig.pdf')
    except:
        pass
    try:
        # http://no-xml.socionet.ru/~cyrcitec/jfmt.cgi?file=j-nbr/nberwo/0011.json
        wget.download(f"{url_prefix}jfmt.cgi?file={journal_link_json[url_prefix_len:]}", 
                      out=f'articles/{journal_id}/{journal_id}:orig.txt')
    except:
        pass
    

34097.xml
None None None
1821.xml
None None None
87577.xml
repec:pra:mprapa:87577 http://no-xml.socionet.ru/~cyrcitec/pdf-cache/mpra.ub.uni-muenchen.de/87577/1/MPRA_paper_87577.pdf http://no-xml.socionet.ru/~cyrcitec/j-pra/mprapa/87577.json
24332.xml
repec:pra:mprapa:24332 http://no-xml.socionet.ru/~cyrcitec/pdf-cache/mpra.ub.uni-muenchen.de/24332/1/MPRA_paper_24332.pdf http://no-xml.socionet.ru/~cyrcitec/j-pra/mprapa/24332.json
13513.xml
repec:pra:mprapa:13513 http://no-xml.socionet.ru/~cyrcitec/pdf-cache/mpra.ub.uni-muenchen.de/13513/1/MPRA_paper_13513.pdf http://no-xml.socionet.ru/~cyrcitec/j-pra/mprapa/13513.json
36366.xml
repec:pra:mprapa:36366 http://no-xml.socionet.ru/~cyrcitec/pdf-cache/mpra.ub.uni-muenchen.de/36366/1/MPRA_paper_36366.pdf http://no-xml.socionet.ru/~cyrcitec/j-pra/mprapa/36366.json
84675.xml
None None None
48076.xml
repec:pra:mprapa:48076 http://no-xml.socionet.ru/~cyrcitec/pdf-cache/mpra.ub.uni-muenchen.de/48076/1/MPRA_paper_48076.pdf http://no-xml.socionet.ru/

84341.xml
58584.xml
repec:pra:mprapa:58584 http://no-xml.socionet.ru/~cyrcitec/pdf-cache/mpra.ub.uni-muenchen.de/58584/1/MPRA_paper_58584.pdf http://no-xml.socionet.ru/~cyrcitec/j-pra/mprapa/58584.json
76094.xml
repec:pra:mprapa:76094 http://no-xml.socionet.ru/~cyrcitec/pdf-cache/mpra.ub.uni-muenchen.de/76094/1/MPRA_paper_76094.pdf http://no-xml.socionet.ru/~cyrcitec/j-pra/mprapa/76094.json
72905.xml
24900.xml
repec:pra:mprapa:24900 http://no-xml.socionet.ru/~cyrcitec/pdf-cache/mpra.ub.uni-muenchen.de/24900/1/MPRA_paper_24900.pdf http://no-xml.socionet.ru/~cyrcitec/j-pra/mprapa/24900.json
64723.xml
repec:pra:mprapa:64723 http://no-xml.socionet.ru/~cyrcitec/pdf-cache/mpra.ub.uni-muenchen.de/64723/9/MPRA_paper_64723.pdf http://no-xml.socionet.ru/~cyrcitec/j-pra/mprapa/64723.json
56993.xml
repec:pra:mprapa:56993 http://no-xml.socionet.ru/~cyrcitec/pdf-cache/mpra.ub.uni-muenchen.de/56993/1/MPRA_paper_56993.pdf http://no-xml.socionet.ru/~cyrcitec/j-pra/mprapa/56993.json
2158.xml
repec:pra:m

51642.xml
repec:pra:mprapa:51642 http://no-xml.socionet.ru/~cyrcitec/pdf-cache/mpra.ub.uni-muenchen.de/51642/1/MPRA_paper_51642.pdf http://no-xml.socionet.ru/~cyrcitec/j-pra/mprapa/51642.json
49073.xml
repec:pra:mprapa:49073 http://no-xml.socionet.ru/~cyrcitec/pdf-cache/mpra.ub.uni-muenchen.de/49073/1/MPRA_paper_49073.pdf http://no-xml.socionet.ru/~cyrcitec/j-pra/mprapa/49073.json
39024.xml
repec:pra:mprapa:39024 http://no-xml.socionet.ru/~cyrcitec/pdf-cache/mpra.ub.uni-muenchen.de/39024/1/MPRA_paper_39024.pdf http://no-xml.socionet.ru/~cyrcitec/j-pra/mprapa/39024.json
31930.xml
repec:pra:mprapa:31930 http://no-xml.socionet.ru/~cyrcitec/pdf-cache/mpra.ub.uni-muenchen.de/31930/1/MPRA_paper_31930.pdf http://no-xml.socionet.ru/~cyrcitec/j-pra/mprapa/31930.json
13727.xml
repec:pra:mprapa:13727 http://no-xml.socionet.ru/~cyrcitec/pdf-cache/mpra.ub.uni-muenchen.de/13727/1/MPRA_paper_13727.pdf http://no-xml.socionet.ru/~cyrcitec/j-pra/mprapa/13727.json
3642.xml
repec:pra:mprapa:3642 http://no

27218.xml
repec:pra:mprapa:27218 http://no-xml.socionet.ru/~cyrcitec/pdf-cache/mpra.ub.uni-muenchen.de/27218/1/MPRA_paper_27218.pdf http://no-xml.socionet.ru/~cyrcitec/j-pra/mprapa/27218.json
55877.xml
None None None
84593.xml
5176.xml
repec:pra:mprapa:5176 http://no-xml.socionet.ru/~cyrcitec/pdf-cache/mpra.ub.uni-muenchen.de/5176/1/MPRA_paper_5176.pdf http://no-xml.socionet.ru/~cyrcitec/j-pra/mprapa/5176.json
409.xml
repec:pra:mprapa:409 http://no-xml.socionet.ru/~cyrcitec/pdf-cache/mpra.ub.uni-muenchen.de/409/1/MPRA_paper_409.pdf http://no-xml.socionet.ru/~cyrcitec/j-pra/mprapa/409.json
51827.xml
repec:pra:mprapa:51827 http://no-xml.socionet.ru/~cyrcitec/pdf-cache/mpra.ub.uni-muenchen.de/51827/1/MPRA_paper_51827.pdf http://no-xml.socionet.ru/~cyrcitec/j-pra/mprapa/51827.json
78422.xml
repec:pra:mprapa:78422 http://no-xml.socionet.ru/~cyrcitec/pdf-cache/mpra.ub.uni-muenchen.de/78422/1/MPRA_paper_78422.pdf http://no-xml.socionet.ru/~cyrcitec/j-pra/mprapa/78422.json
32520.xml
repec:pra:

65204.xml
repec:pra:mprapa:65204 http://no-xml.socionet.ru/~cyrcitec/pdf-cache/mpra.ub.uni-muenchen.de/65204/1/MPRA_paper_65204.pdf http://no-xml.socionet.ru/~cyrcitec/j-pra/mprapa/65204.json
27600.xml
None None None
73275.xml
repec:pra:mprapa:73275 http://no-xml.socionet.ru/~cyrcitec/pdf-cache/mpra.ub.uni-muenchen.de/73275/1/MPRA_paper_73275.pdf http://no-xml.socionet.ru/~cyrcitec/j-pra/mprapa/73275.json
12010.xml
repec:pra:mprapa:12010 http://no-xml.socionet.ru/~cyrcitec/pdf-cache/mpra.ub.uni-muenchen.de/12010/1/MPRA_paper_12010.pdf http://no-xml.socionet.ru/~cyrcitec/j-pra/mprapa/12010.json
5659.xml
repec:pra:mprapa:5659 http://no-xml.socionet.ru/~cyrcitec/pdf-cache/mpra.ub.uni-muenchen.de/5659/1/MPRA_paper_5659.pdf http://no-xml.socionet.ru/~cyrcitec/j-pra/mprapa/5659.json
88122.xml
repec:pra:mprapa:88122 http://no-xml.socionet.ru/~cyrcitec/pdf-cache/mpra.ub.uni-muenchen.de/88122/1/MPRA_paper_88122.pdf http://no-xml.socionet.ru/~cyrcitec/j-pra/mprapa/88122.json
35621.xml
None None 